In [1]:
!pip install --upgrade pip
!pip install pymc arviz numpy scipy scikit-learn

In [2]:
!pip install ucimlrepo
!pip install optuna

In [3]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
import pymc as pm
#import aesara
#import aesara.tensor as at
from sklearn.metrics import accuracy_score, log_loss, f1_score

import numpy as np
from sklearn.base import clone
from scipy.stats import norm, gaussian_kde
from sklearn.metrics import accuracy_score, log_loss
from sklearn.base import clone
import numpy as np
from sklearn.base import clone
from sklearn.metrics import accuracy_score, log_loss
from sklearn.base import BaseEstimator, ClassifierMixin, clone


class EstimatorWithCoef(BaseEstimator, ClassifierMixin):
    """
    Wraps a Bayesian logistic estimator so that after fit() it exposes:
      - coef_  (pulls from coef_ if present, else from coef_mean_)
      - predict / predict_proba / score delegated to the inner model
    This makes it compatible with sklearn RFE.
    """
    def __init__(self, base_estimator):
        self.base_estimator = base_estimator
        self.model_ = None
        self.coef_ = None

    def get_params(self, deep=True):
        # So GridSearchCV/clone works
        return {"base_estimator": self.base_estimator}

    def set_params(self, **params):
        if "base_estimator" in params:
            self.base_estimator = params["base_estimator"]
        return self

    def fit(self, X, y):
        self.model_ = clone(self.base_estimator).fit(X, y)
        if hasattr(self.model_, "coef_"):
            self.coef_ = getattr(self.model_, "coef_")
        elif hasattr(self.model_, "coef_mean_"):
            self.coef_ = getattr(self.model_, "coef_mean_")
        else:
            raise AttributeError("Wrapped estimator must expose coef_ or coef_mean_ after fit().")
        # Flatten shapes like (1,p)
        self.coef_ = getattr(self.coef_, "ravel", lambda: self.coef_)()
        return self

    def predict(self, X):
        return self.model_.predict(X)

    def predict_proba(self, X):
        return self.model_.predict_proba(X)

    def score(self, X, y):
        # default classifier score = accuracy
        return self.model_.score(X, y)


from mlxtend.feature_selection import SequentialFeatureSelector as MLX_SFS
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, f1_score
from sklearn.base import clone
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.feature_selection import RFE, SequentialFeatureSelector
from sklearn.base import clone
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Custom scorer

macro_f1 = make_scorer(f1_score, average="macro")


from sklearn.model_selection import StratifiedShuffleSplit

def rfe_select_features(X_train, y_train, **kwargs):
    estimator = kwargs.get('estimator', LogisticRegression(solver='liblinear'))

    if 'k' in kwargs:
        k = kwargs['k']
        rfe = RFE(estimator, n_features_to_select=k)
        rfe.fit(X_train, y_train)
        selected = np.where(rfe.support_)[0]
        return selected, None

    else:
        min_k = kwargs.get('min_k', 1)
        max_k = kwargs.get('max_k', X_train.shape[1])
        n_splits = kwargs.get('n_splits', 2)
        test_size = kwargs.get('test_size', 0.2)
        random_state = kwargs.get('random_state', 42)

        cv = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)

        best_score = -np.inf
        best_k = 1
        no_improve_counter = 0
        patience = 3

        for k in range(min_k, max_k + 1, 1):
            rfe = RFE(estimator, n_features_to_select=k)
            X_trans = rfe.fit_transform(X_train, y_train)
            score = np.mean(cross_val_score(clone(estimator), X_trans, y_train, cv=cv, scoring=macro_f1, n_jobs=-1))

            if score > best_score:
                best_score = score
                best_k = k
                no_improve_counter = 0
            else:
                no_improve_counter += 1
                if no_improve_counter >= patience:
                    break

        rfe = RFE(estimator, n_features_to_select=best_k)
        rfe.fit(X_train, y_train)
        selected = np.where(rfe.support_)[0]
        return selected, None

def _get_cv_splitter(cv, X, y):
    if isinstance(cv, int):
        return StratifiedShuffleSplit(n_splits=cv, test_size=0.2, random_state=42)
    return cv

def sfs_select_features(X_train, y_train, **kwargs):
    estimator = kwargs.get('estimator')
    cv_folds = _get_cv_splitter(kwargs.get('cv', 2), X_train, y_train)

    if 'k' in kwargs:
        k = kwargs['k']
        sfs = MLX_SFS(clone(estimator),
                      k_features=k,
                      forward=True,
                      floating=False,
                      scoring=macro_f1,
                      cv=cv_folds,
                      n_jobs=-1)
        sfs.fit(X_train, y_train)
        selected = np.array(sfs.k_feature_idx_)
        return selected, None
    else:
        min_k = kwargs.get('min_k', 1)
        max_k = kwargs.get('max_k', X_train.shape[1])
        best_score = -np.inf
        best_support = None
        no_improve_counter = 0
        patience = 3

        for k in range(min_k, max_k + 1, 1):
            sfs = MLX_SFS(clone(estimator),
                          k_features=k,
                          forward=True,
                          floating=False,
                          scoring=macro_f1,
                          cv=cv_folds,
                          n_jobs=-1)
            sfs.fit(X_train, y_train)
            if sfs.k_score_ > best_score:
                best_score = sfs.k_score_
                best_support = sfs.k_feature_idx_
                no_improve_counter = 0
            else:
                no_improve_counter += 1
                if no_improve_counter >= patience:
                    break

        selected = np.array(best_support)
        return selected, None

def sbs_select_features(X_train, y_train, **kwargs):
    estimator = kwargs.get('estimator')
    cv_folds = _get_cv_splitter(kwargs.get('cv', 2), X_train, y_train)

    if 'k' in kwargs:
        k = kwargs['k']
        sbs = MLX_SFS(clone(estimator),
                      k_features=k,
                      forward=False,
                      floating=False,
                      scoring=macro_f1,
                      cv=cv_folds,
                      n_jobs=-1)
        sbs.fit(X_train, y_train)
        selected = np.array(sbs.k_feature_idx_)
        return selected, None
    else:
        min_k = kwargs.get('min_k', 1)
        max_k = kwargs.get('max_k', X_train.shape[1])
        best_score = -np.inf
        best_support = None
        no_improve_counter = 0
        patience = 3

        #for k in range(min_k, max_k + 1, 5):
        for k in reversed(range(min_k, max_k + 1, 1)):
            sbs = MLX_SFS(clone(estimator),
                          k_features=k,
                          forward=False,
                          floating=False,
                          scoring=macro_f1,
                          cv=cv_folds,
                          n_jobs=-1)
            sbs.fit(X_train, y_train)
            if sbs.k_score_ > best_score:
                best_score = sbs.k_score_
                best_support = sbs.k_feature_idx_
                no_improve_counter = 0
            else:
                no_improve_counter += 1
                if no_improve_counter >= patience:
                    break

        selected = np.array(best_support)
        return selected, None

def bidir_select_features(X_train, y_train, **kwargs):
    estimator = kwargs.get('estimator')
    cv_folds = _get_cv_splitter(kwargs.get('cv', 2), X_train, y_train)

    if 'k' in kwargs:
        k = kwargs['k']
        bidi = MLX_SFS(clone(estimator),
                       k_features=k,
                       forward=True,
                       floating=True,
                       scoring=macro_f1,
                       cv=cv_folds,
                       n_jobs=-1)
        bidi.fit(X_train, y_train)
        selected = np.array(bidi.k_feature_idx_)
        return selected, None
    else:
        min_k = kwargs.get('min_k', 1)
        max_k = kwargs.get('max_k', X_train.shape[1])
        best_score = -np.inf
        best_support = None
        no_improve_counter = 0
        patience = 3

        for k in range(min_k, max_k + 1, 1):
            bidi = MLX_SFS(clone(estimator),
                           k_features=k,
                           forward=True,
                           floating=True,
                           scoring=macro_f1,
                           cv=cv_folds,
                           n_jobs=-1)
            bidi.fit(X_train, y_train)
            if bidi.k_score_ > best_score:
                best_score = bidi.k_score_
                best_support = bidi.k_feature_idx_
                no_improve_counter = 0
            else:
                no_improve_counter += 1
                if no_improve_counter >= patience:
                    break

        selected = np.array(best_support)
        return selected, None


def rfe_select_features_old(X_train, y_train, **kwargs):
    estimator = kwargs.get('estimator', LogisticRegression(solver='liblinear'))

    if 'k' in kwargs:
        k = kwargs['k']
        rfe = RFE(estimator, n_features_to_select=k)
        rfe.fit(X_train, y_train)
        selected = np.where(rfe.support_)[0]
        return selected, None

    else:
        max_k = kwargs.get('max_k', X_train.shape[1])
        n_splits = kwargs.get('n_splits', 2)
        test_size = kwargs.get('test_size', 0.2)
        random_state = kwargs.get('random_state', 42)

        cv = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)

        best_score = -np.inf
        best_k = 1

        for k in range(1, max_k + 1, 5):
            rfe = RFE(estimator, n_features_to_select=k)
            X_trans = rfe.fit_transform(X_train, y_train)
            score = np.mean(cross_val_score(clone(estimator), X_trans, y_train, cv=cv, scoring=macro_f1, n_jobs=-1))

            if score > best_score:
                best_score = score
                best_k = k

        # Final fit with best_k
        rfe = RFE(estimator, n_features_to_select=best_k)
        rfe.fit(X_train, y_train)
        selected = np.where(rfe.support_)[0]
        return selected, None


def _get_cv_splitter(cv, X, y):
    if isinstance(cv, int):
        return StratifiedShuffleSplit(n_splits=cv, test_size=0.2, random_state=42)
    return cv


def sfs_select_features_old(X_train, y_train, **kwargs):
    estimator = kwargs.get('estimator')
    cv_folds = _get_cv_splitter(kwargs.get('cv', 2), X_train, y_train)

    if 'k' in kwargs:
        k = kwargs['k']
        sfs = MLX_SFS(clone(estimator),
                      k_features=k,
                      forward=True,
                      floating=False,
                      scoring=macro_f1,
                      cv=cv_folds,
                      n_jobs=-1)
        sfs.fit(X_train, y_train)
        selected = np.array(sfs.k_feature_idx_)
        return selected, None
    else:
        min_k = kwargs.get('min_k', 1)
        max_k = kwargs.get('max_k', X_train.shape[1])
        best_score = -np.inf
        best_support = None

        for k in range(min_k, max_k + 1, 5):
            sfs = MLX_SFS(clone(estimator),
                          k_features=k,
                          forward=True,
                          floating=False,
                          scoring=macro_f1,
                          cv=cv_folds,
                          n_jobs=-1)
            sfs.fit(X_train, y_train)
            if sfs.k_score_ > best_score:
                best_score = sfs.k_score_
                best_support = sfs.k_feature_idx_

        selected = np.array(best_support)
        return selected, None


def sbs_select_features_old(X_train, y_train, **kwargs):
    estimator = kwargs.get('estimator')
    cv_folds = _get_cv_splitter(kwargs.get('cv', 2), X_train, y_train)

    if 'k' in kwargs:
        k = kwargs['k']
        sbs = MLX_SFS(clone(estimator),
                      k_features=k,
                      forward=False,
                      floating=False,
                      scoring=macro_f1,
                      cv=cv_folds,
                      n_jobs=-1)
        sbs.fit(X_train, y_train)
        selected = np.array(sbs.k_feature_idx_)
        return selected, None
    else:
        min_k = kwargs.get('min_k', 1)
        max_k = kwargs.get('max_k', X_train.shape[1])
        best_score = -np.inf
        best_support = None

        for k in range(min_k, max_k + 1, 5):
            sbs = MLX_SFS(clone(estimator),
                          k_features=k,
                          forward=False,
                          floating=False,
                          scoring=macro_f1,
                          cv=cv_folds,
                          n_jobs=-1)
            sbs.fit(X_train, y_train)
            if sbs.k_score_ > best_score:
                best_score = sbs.k_score_
                best_support = sbs.k_feature_idx_

        selected = np.array(best_support)
        return selected, None


def bidir_select_features_old(X_train, y_train, **kwargs):
    estimator = kwargs.get('estimator')
    cv_folds = _get_cv_splitter(kwargs.get('cv', 2), X_train, y_train)

    if 'k' in kwargs:
        k = kwargs['k']
        bidi = MLX_SFS(clone(estimator),
                       k_features=k,
                       forward=True,
                       floating=True,
                       scoring=macro_f1,
                       cv=cv_folds,
                       n_jobs=-1)
        bidi.fit(X_train, y_train)
        selected = np.array(bidi.k_feature_idx_)
        return selected, None
    else:
        min_k = kwargs.get('min_k', 1)
        max_k = kwargs.get('max_k', X_train.shape[1])
        best_score = -np.inf
        best_support = None

        for k in range(min_k, max_k + 1, 5):
            bidi = MLX_SFS(clone(estimator),
                           k_features=k,
                           forward=True,
                           floating=True,
                           scoring=macro_f1,
                           cv=cv_folds,
                           n_jobs=-1)
            bidi.fit(X_train, y_train)
            if bidi.k_score_ > best_score:
                best_score = bidi.k_score_
                best_support = bidi.k_feature_idx_

        selected = np.array(best_support)
        return selected, None



def _to_index_or_mask(selected_idx, n_features):
    """
    Normalize selector output into either:
      - an integer index array (dtype=int), or
      - a boolean mask of length n_features.
    """
    # None or empty -> keep all
    if selected_idx is None:
        return np.arange(n_features, dtype=int)
    # convert sets/tuples/lists to np array
    arr = np.asarray(list(selected_idx) if isinstance(selected_idx, set) else selected_idx)

    # boolean mask?
    if arr.dtype == bool:
        if arr.ndim != 1 or arr.shape[0] != n_features:
            raise ValueError("Boolean mask has wrong shape.")
        return arr  # mask

    # tuple from mlxtend (k_feature_idx_) can be tuple of ints
    # also handle nested tuples/lists (e.g. ((1,3,4,5,8),))
    if arr.ndim > 1:
        arr = np.ravel(arr)

    # cast to int safely
    try:
        idx = arr.astype(int)
    except Exception:
        # last resort: map each entry to int explicitly
        idx = np.array([int(x) for x in arr])

    # guard for NaNs or negatives
    if np.any(idx < 0) or np.any(idx >= n_features):
        raise IndexError(f"Selected indices out of bounds for n_features={n_features}: {idx}")
    return idx


def evaluate_feature_selection_method_cls(feature_selector,
                                          X_train, y_train,
                                          X_test, y_test,
                                          selector_params={},
                                          model_params={},
                                          eval_indices_bs=None):
    """
    Evaluate feature selection for classification using Bayesian models.

    - Selects features on Train
    - Fits Bayesian model on Train-selected features
    - Reports Accuracy, Macro-F1, Log-loss on Test
    - Computes Sign Entropy from posterior (mu, sigma) of Train-fitted model
    """

    #selector_params = selector_params.copy()  # avoid mutating original
    est = clone(selector_params['evaluator'])
    est_test = clone(selector_params['evaluator'])

    evaluator = selector_params.pop('evaluator', None)

    # run feature selector
    selected_idx, _ = feature_selector(X_train, y_train, **selector_params)

    # normalize selection
    n_features = X_train.shape[1]
    sel = _to_index_or_mask(selected_idx, n_features)

    # slice data
    if isinstance(sel, np.ndarray) and sel.dtype == bool:
        X_tr, X_te = X_train[:, sel], X_test[:, sel]
        kept_idx = np.where(sel)[0]
    else:
        X_tr, X_te = X_train[:, sel], X_test[:, sel]
        kept_idx = np.asarray(sel, dtype=int)

    # fit estimator on Train
    #est = clone(selector_params['evaluator']).fit(X_tr, y_train)
    est.fit(X_tr, y_train)

    # predictions on Test
    if hasattr(est, "predict_proba"):
        proba = est.predict_proba(X_te)
        if proba.ndim == 1 or proba.shape[1] == 1:
            p1 = np.ravel(proba); proba = np.c_[1 - p1, p1]
        y_pred = (proba[:, 1] >= 0.5).astype(int)
        acc = accuracy_score(y_test, y_pred)
        macro_f1 = f1_score(y_test, y_pred, average="macro")
        ll  = log_loss(y_test, proba)
    else:
        y_pred = est.predict(X_te)
        acc = accuracy_score(y_test, y_pred)
        macro_f1 = f1_score(y_test, y_pred, average="macro")
        ll  = None

    # stability: sign entropy from posterior (mu, sigma) in test
    #est_test = clone(selector_params['evaluator']).fit(X_te, y_te)
    est_test.fit(X_te, y_te)

    entropies = None
    if hasattr(est_test, "coef_") and hasattr(est_test, "sigma_"):
        # Bayesian evaluator: direct posterior mean + std
        entropies = calculate_entropy_dist(est_test.coef_, est_test.sigma_)
    else:
        # Frequentist evaluator: bootstrap 1000 fits on the *test set*
        n_bootstraps = 1000
        coefs = []

        rng = np.random.RandomState(42)
        for _ in range(n_bootstraps):
            # stratified bootstrap indices
            classes, counts = np.unique(y_te, return_counts=True)
            idxs = []
            for c, cnt in zip(classes, counts):
                cls_idx = np.where(y_te == c)[0]
                take = rng.choice(cls_idx, size=cnt, replace=True)
                idxs.append(take)
            idxs = np.concatenate(idxs)
            rng.shuffle(idxs)

            X_bs, y_bs = X_te[idxs], y_te[idxs]
            if len(np.unique(y_bs)) < 2:
                continue  # skip if only one class

            try:
                clf_bs = clone(evaluator).fit(X_bs, y_bs)
                coefs.append(np.ravel(clf_bs.coef_))
            except Exception:
                continue

        if len(coefs) < 10:
            # too few successes -> signal with NaNs
            entropies = np.full(X_te.shape[1], np.nan)
        else:
            coef_mat = np.vstack(coefs)        # shape: (n_bootstraps, p)
            entropies = calculate_entropies(coef_mat)  # your KDE-based fn

        entropies = np.atleast_1d(np.asarray(entropies, dtype=float))

    return kept_idx, {
        "accuracy": acc,
        "macro_f1": macro_f1,
        "log_loss": ll,
        "sign_entropy": entropies,
        "mean_entropy": np.mean(entropies) if entropies is not None else None
    }


# --- helpers ---
def calculate_entropy(data):
    if np.var(data) == 0:
        return 0

    scipy_kernel = gaussian_kde(data)

    #  We calculate the bandwidth for later use
    optimal_bandwidth = scipy_kernel.factor * np.std(data)

    # Calculate KDE for the entire dataset
    kde = gaussian_kde(data, bw_method=optimal_bandwidth)

    # Create a range of values to represent the KDE
    x = np.linspace(np.min(data), np.max(data), 1000)

    # Evaluate the density at each point in the range
    density = kde(x)

    # Normalize the density function
    normalized_density = density / np.sum(density * (x[1] - x[0]))

    # Calculate the probabilities of positive and negative values
    positive_probability = np.sum(normalized_density[x >= 0] * (x[1] - x[0]))
    negative_probability = np.sum(normalized_density[x < 0] * (x[1] - x[0]))

    if positive_probability == 0 or negative_probability == 0:
        sign_entropy = 0
    else:
        sign_entropy = -positive_probability * np.log2(positive_probability) \
                       - negative_probability * np.log2(negative_probability)

    return sign_entropy


def calculate_entropies(result_matrix):
    sign_entropies = []
    for column in range(result_matrix.shape[1]):
        data = result_matrix[:, column]
        sign_entropy = calculate_entropy(data)
        sign_entropies.append(sign_entropy)

    sign_entropies = np.array(sign_entropies)

    return sign_entropies


def _entropy_from_samples(beta_samples):
    # beta_samples: (draws, p) posterior samples (no intercept)
    p_pos = (beta_samples > 0).mean(axis=0)
    p_pos = np.clip(p_pos, 1e-12, 1 - 1e-12)  # avoid log(0)
    p_neg = 1.0 - p_pos
    return -(p_pos * np.log2(p_pos) + p_neg * np.log2(p_neg))


def _entropy_from_gaussian(mu, Sigma, has_intercept=False):
    # mu: (p,) or (p+1,) ; Sigma: (p,p) or (p+1,p+1) or diag vector
    mu = np.ravel(mu)
    if Sigma.ndim == 2:
        diag_vars = np.diag(Sigma)
    else:
        diag_vars = np.ravel(Sigma)
    if has_intercept:
        mu = mu[1:]
        diag_vars = diag_vars[1:]
    sigma = np.sqrt(np.clip(diag_vars, 1e-12, None))
    p_neg = norm.cdf(0.0, loc=mu, scale=sigma)
    p_pos = 1.0 - p_neg
    H = np.zeros_like(mu, dtype=float)
    mid = (p_pos > 0) & (p_pos < 1)
    H[mid] = -(p_pos[mid]*np.log2(p_pos[mid]) + p_neg[mid]*np.log2(p_neg[mid]))
    return H


# ---------- utilities ----------
def calculate_entropy_from_samples(beta_samples):
    """
    beta_samples: array (n_draws, p) of posterior draws for coefficients (no intercept).
    Return: entropy per feature (p,), using base-2 logs.
    """
    p_pos = (beta_samples > 0).mean(axis=0)
    p_pos = np.clip(p_pos, 1e-12, 1 - 1e-12)  # numerical safety at boundaries
    p_neg = 1.0 - p_pos
    H = -(p_pos * np.log2(p_pos) + p_neg * np.log2(p_neg))
    return H

# ---------- Bayesian logistic classifier (PyMC, ADVI for speed) ----------
class BayesianLogisticPyMC_old(BaseEstimator, ClassifierMixin):
    """
    Bayesian logistic regression using PyMC with Normal(0, tau) priors and Bernoulli likelihood.
    Uses ADVI (variational inference) to approximate the posterior quickly.

    Attributes after fit():
      coef_samples_ : (n_draws, p)  posterior draws for coefficients (no intercept)
      intercept_samples_ : (n_draws,) posterior draws for intercept
      coef_mean_ : (p,) mean of posterior draws
      Sigma_ : (p,p) empirical covariance of posterior draws (useful if needed)
    """
    def __init__(self, tau=0.1, fit_intercept=True, advi_iters=20000, draws=2000, random_state=0, verbose=False):
        self.tau = float(tau)  # prior std; prior is Normal(0, tau)
        self.fit_intercept = bool(fit_intercept)
        self.advi_iters = int(advi_iters)
        self.draws = int(draws)
        self.random_state = int(random_state)
        self.verbose = bool(verbose)

        # learned
        self.coef_samples_ = None
        self.intercept_samples_ = None
        self.coef_mean_ = None
        self.Sigma_ = None

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=int).ravel()
        if set(np.unique(y)) - {0, 1}:
            raise ValueError("Binary labels {0,1} required.")

        n, p = X.shape
        rng = np.random.default_rng(self.random_state)

        with pm.Model() as model:
            beta = pm.Normal("beta", mu=0.0, sigma=self.tau, shape=p)
            if self.fit_intercept:
                alpha = pm.Normal("alpha", mu=0.0, sigma=10.0)  # weak prior for intercept
                eta = alpha + pm.math.dot(X, beta)
            else:
                eta = pm.math.dot(X, beta)

            p1 = pm.math.sigmoid(eta)
            pm.Bernoulli("y", p=p1, observed=y)

            # Variational inference (ADVI) for speed; returns an approx posterior
            approx = pm.fit(self.advi_iters, method="advi", random_seed=self.random_state, progressbar=self.verbose)
            idata = approx.sample(self.draws, random_seed=self.random_state)

        # Extract posterior draws
        beta_draws = np.asarray(idata.posterior["beta"]).reshape(-1, p)  # (chains*draws, p)
        self.coef_samples_ = beta_draws
        if self.fit_intercept:
            alpha_draws = np.asarray(idata.posterior["alpha"]).reshape(-1)
            self.intercept_samples_ = alpha_draws

        # Posterior summaries
        self.coef_mean_ = beta_draws.mean(axis=0)
        self.Sigma_ = np.cov(beta_draws.T)  # empirical covariance
        self.coef_ = self.coef_mean_
        self.sigma_ = np.diag(self.Sigma_)  # or full matrix if you prefer

        return self


    def predict_proba(self, X):
        X = np.asarray(X, dtype=float)
        # MC average of predictive probabilities over posterior draws
        eta_samples = X @ self.coef_samples_.T  # (n, draws)
        if self.fit_intercept and self.intercept_samples_ is not None:
            eta_samples = eta_samples + self.intercept_samples_[None, :]
        p1_samples = 1.0 / (1.0 + np.exp(-eta_samples))
        p1_mean = p1_samples.mean(axis=1)
        return np.column_stack([1.0 - p1_mean, p1_mean])

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)


class BayesianLogisticPyMC(BaseEstimator, ClassifierMixin):
    """
    Bayesian logistic regression with Normal(0, prior_std) priors on weights.
    NOTE: 'prior_std' is a STANDARD DEVIATION, not a precision.
    """

    def __init__(self, prior_std=1.0, fit_intercept=True,
                 advi_iters=20000, draws=2000, random_state=0, verbose=False,
                 penalize_intercept=False, intercept_std=10.0,  # if True, Normal(0, intercept_std); else Flat
                 fullrank=False):
        self.prior_std = float(prior_std)
        self.fit_intercept = bool(fit_intercept)
        self.advi_iters = int(advi_iters)
        self.draws = int(draws)
        self.random_state = int(random_state)
        self.verbose = bool(verbose)
        self.penalize_intercept = bool(penalize_intercept)
        self.intercept_std = float(intercept_std)
        self.fullrank = bool(fullrank)

        self.coef_samples_ = None
        self.intercept_samples_ = None
        self.coef_mean_ = None
        self.Sigma_ = None
        self.coef_ = None
        self.sigma_ = None  # per-feature posterior std

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=int).ravel()
        if set(np.unique(y)) - {0, 1}:
            raise ValueError("Binary labels {0,1} required.")
        n, p = X.shape

        with pm.Model() as model:
            beta = pm.Normal("beta", mu=0.0, sigma=self.prior_std, shape=p)

            if self.fit_intercept:
                if self.penalize_intercept:
                    alpha = pm.Normal("alpha", mu=0.0, sigma=self.intercept_std)
                else:
                    alpha = pm.Flat("alpha")  # unpenalized intercept, like sklearn
                eta = alpha + pm.math.dot(X, beta)
            else:
                eta = pm.math.dot(X, beta)

            pm.Bernoulli("y", p=pm.math.sigmoid(eta), observed=y)

            method = "fullrank_advi" if self.fullrank else "advi"
            approx = pm.fit(self.advi_iters, method=method,
                            random_seed=self.random_state, progressbar=self.verbose)
            idata = approx.sample(self.draws, random_seed=self.random_state)

        # Extract posterior draws
        beta_draws = np.asarray(idata.posterior["beta"]).reshape(-1, p)
        self.coef_samples_ = beta_draws
        if self.fit_intercept:
            self.intercept_samples_ = np.asarray(idata.posterior["alpha"]).reshape(-1)

        # Summaries
        # Posterior summaries (robust to p==1)
        self.coef_mean_ = beta_draws.mean(axis=0)
        self.sigma_     = beta_draws.std(axis=0, ddof=1)
        p = beta_draws.shape[1]
        if p > 1:
            self.Sigma_ = np.cov(beta_draws.T, ddof=1)
        else:
            var = float(self.sigma_[0] ** 2)
            self.Sigma_ = np.array([[var]])
        self.coef_ = self.coef_mean_

        return self

    def predict_proba(self, X):
        X = np.asarray(X, dtype=float)
        eta = X @ self.coef_samples_.T
        if self.fit_intercept and self.intercept_samples_ is not None:
            eta = eta + self.intercept_samples_[None, :]
        p1 = 1.0 / (1.0 + np.exp(-eta))
        p1_mean = p1.mean(axis=1)
        return np.column_stack([1.0 - p1_mean, p1_mean])

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] >= 0.5).astype(int)


# ---------- Bay-CoFE selector for classification ----------

from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
import numpy as np

def sefe_select_features(X_train, y_train, **kwargs):
    """
    SEFE for classification using sklearn LogisticRegression and bootstrapped sign entropy:
    Iteratively fit a logistic regression estimator and drop features
    whose sign entropy > slack, estimated via KDE on bootstrapped coefs.

    kwargs:
      slack (float)     : entropy threshold (default 0.001)
      num_iter (int)    : max elimination passes (default 10)
      tolerance_limit   : stop if no removals for this many consecutive passes (default 1)
      n_bootstraps      : number of bootstraps for sign entropy estimation (default 1000)

    returns:
      np.ndarray of kept feature indices (0-based)
    """
    slack = float(kwargs.get('slack', 0.01))
    num_iter = int(kwargs.get('num_iter', 10))
    tolerance_limit = int(kwargs.get('tolerance_limit', 3))
    n_bootstraps = int(kwargs.get('n_bootstraps', 1000))

    base_estimator = kwargs.get('estimator')

    p = X_train.shape[1]
    drop_mask = np.zeros(p, dtype=bool)
    patience = 0

    for _ in range(num_iter):
        keep_idx = np.where(~drop_mask)[0]
        if keep_idx.size == 0:
            break

        X_sel = X_train[:, keep_idx]
        boot_coefs = []

        for _ in range(n_bootstraps):
            indices = np.random.choice(X_sel.shape[0], size=X_sel.shape[0], replace=True)
            X_bs = X_sel[indices]
            y_bs = y_train[indices]

            clf = clone(base_estimator)
            try:
                clf.fit(X_bs, y_bs)
                coefs = np.ravel(clf.coef_)
                boot_coefs.append(coefs)
            except Exception:
                continue

        boot_coefs = np.array(boot_coefs)
        if boot_coefs.shape[0] < 10:
            raise RuntimeError("Too few successful bootstraps — consider using a more stable estimator.")

        # Compute per-feature sign entropy
        H = calculate_entropies(boot_coefs)
        H = np.atleast_1d(np.asarray(H, dtype=float))

        # Drop features with high sign entropy
        to_drop_local = to_drop_local = np.flatnonzero(np.isfinite(H) & (H > slack)) #np.where(H > slack)[0]
        if to_drop_local.size > 0:
            drop_mask[keep_idx[to_drop_local]] = True
            patience = 0
        else:
            patience += 1
            if patience >= tolerance_limit:
                break
        #print(drop_mask)

    selected = np.where(~drop_mask)[0]
    if selected.size == 0:
        selected = np.arange(p)  # fallback: keep all
    return selected, None


def calculate_entropy_dist(mu, sigma):
    """
    Vectorized sign-entropy for Gaussian (mu, sigma).
    Handles scalars and 1-D arrays cleanly.
    """
    mu = np.ravel(mu).astype(float)         # force 1‑D
    sigma = np.ravel(sigma).astype(float)   # force 1‑D
    #print("mu: ", mu)
    #print("sigma: ", sigma)
    p_neg = norm.cdf(0.0, loc=mu, scale=sigma)
    p_pos = 1.0 - p_neg

    # scalar case
    if mu.size == 1:
        if p_pos[0] == 0.0:
            return -(p_neg[0] * np.log2(p_neg[0])) if p_neg[0] > 0 else 0.0
        elif p_neg[0] == 0.0:
            return -(p_pos[0] * np.log2(p_pos[0])) if p_pos[0] > 0 else 0.0
        else:
            return -(p_pos[0] * np.log2(p_pos[0]) + p_neg[0] * np.log2(p_neg[0]))

    # vector case
    H = np.zeros_like(mu, dtype=float)

    mid = (p_pos > 0) & (p_pos < 1)
    H[mid] = -(p_pos[mid] * np.log2(p_pos[mid]) + p_neg[mid] * np.log2(p_neg[mid]))

    zero_pos = (p_pos == 0) & (p_neg > 0)
    H[zero_pos] = -(p_neg[zero_pos] * np.log2(p_neg[zero_pos]))

    zero_neg = (p_neg == 0) & (p_pos > 0)
    H[zero_neg] = -(p_pos[zero_neg] * np.log2(p_pos[zero_neg]))

    return H


def baycofe_select_features_cls(X_train, y_train, **kwargs):
    """
    Iteratively fits the Bayesian logistic model; at each step removes features whose posterior sign entropy > slack.
    Uses posterior mean and covariance (mu, Sigma), not samples.

    Returns:
      selected_idx : indices of kept features
      entropies_last : per-feature sign entropy for the final subset
    """
    slack = float(kwargs.get('slack', 0.01))
    num_iter = int(kwargs.get('num_iter', 10))
    tolerance_limit = int(kwargs.get('tolerance_limit', 3))
    n_bootstraps = int(kwargs.get('n_bootstraps', 1000))

    estimator = kwargs.get('estimator')

    p = X_train.shape[1]
    drop_mask = np.zeros(p, dtype=bool)
    patience = 0
    H_last = np.zeros(p, dtype=float)  # ensure it always exists

    for _ in range(num_iter):
        keep_idx = np.where(~drop_mask)[0]
        if keep_idx.size == 0:
            break

        X_sel = X_train[:, keep_idx]
        clf = clone(estimator).fit(X_sel, y_train)

        H = None
        if hasattr(clf, "coef_") and hasattr(clf, "sigma_"):
          H = calculate_entropy_dist(clf.coef_, clf.sigma_)
          H = np.atleast_1d(np.asarray(H, dtype=float))

        to_drop_local = np.flatnonzero(np.isfinite(H) & (H > slack)) #np.where(H > slack)[0]
        if to_drop_local.size > 0:
            drop_mask[keep_idx[to_drop_local]] = True
            patience = 0
        else:
            patience += 1
            if patience >= tolerance_limit:
                break

    selected_idx = np.where(~drop_mask)[0]
    if selected_idx.size == 0:
        selected_idx = np.arange(p)
        ent_last = np.zeros(p, dtype=float)
    else:
        X_sel = X_train[:, selected_idx]
        clf = clone(estimator).fit(X_sel, y_train)

        if hasattr(clf, "coef_") and hasattr(clf, "sigma_"):
          H_last = calculate_entropy_dist(clf.coef_, clf.sigma_)

    return selected_idx, H_last

In [ ]:
clf = clone(bayes_logit).fit(X_tr, y_tr)
calculate_entropy_dist(clf.coef_, clf.sigma_)

mu:  [-0.27566714  0.76505141  0.42085086 -0.20820214  0.23491621 -0.67149807
 -0.13194625  0.24349458 -1.46157928 -3.76084435 -0.68095906 -2.10793685
 -0.53766211 -0.24357927]
sigma:  [0.11849373 0.09469249 0.10975324 0.06954828 0.06681862 0.10359108
 0.10342454 0.09632291 0.08281227 0.33919111 0.1047588  0.22335525
 0.08820809 0.09164714]


array([ 8.07789604e-02,  1.72334977e-14,  9.68683382e-04,  1.50856872e-02,
        2.98157617e-03,  1.61841337e-09,  4.72213882e-01,  5.09704415e-02,
       -0.00000000e+00, -0.00000000e+00,  1.44251987e-09, -0.00000000e+00,
        1.75845955e-08,  3.70857513e-02])

In [4]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np


def remove_highly_correlated_features_pd(X, threshold):
    # Convert X to a pandas DataFrame
    # Calculate the correlation matrix
    df = pd.DataFrame(X)
    corr_matrix = df.corr().abs()
    # Create a mask to remove the upper triangle of the correlation matrix
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

    # Set the upper triangle values to NaN
    corr_matrix.mask(mask, inplace=True)

    # Find the highly correlated features
    cols_to_drop = [column for column in corr_matrix.columns if any(corr_matrix[column] > threshold)]

    # Drop the highly correlated features from the DataFrame
    reduced_df = df.drop(columns=cols_to_drop)

    # Convert the reduced DataFrame back to numpy array
    X_reduced = reduced_df.to_numpy()

    return X_reduced, cols_to_drop


def get_breast_cancer_XY(threshold=0.80):
    """
    Load Breast Cancer Wisconsin dataset (binary classification).

    Returns:
        X (np.ndarray): scaled feature matrix
        y (np.ndarray): binary labels (0 = malignant, 1 = benign)
    """
    # Load dataset
    data = load_breast_cancer()
    X = pd.DataFrame(data.data, columns=data.feature_names)
    y = data.target  # already 0/1

    # Remove highly correlated features
    X_reduced, removed_features = remove_highly_correlated_features_pd(X, threshold)
    print("Removed due to correlation:", removed_features)

    # Scale features (min–max scaling)
    X_min = X_reduced.min(axis=0)
    X_max = X_reduced.max(axis=0)
    X_scaled = (X_reduced - X_min) / (X_max - X_min)

    # Ensure numpy array
    if isinstance(X_scaled, pd.DataFrame):
        X_scaled = X_scaled.to_numpy()

    print("Shape of X:", X_scaled.shape)
    print("Shape of y:", y.shape)

    return X_scaled, y


# pip install ucimlrepo pandas numpy scikit-learn
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def get_phishing_XY(threshold=0.80, scale="standard"):
    """
    UCI Phishing Websites (binary; ~11k rows, 30 int features).
    scale: 'standard' | 'minmax' | None
    Returns: X (ndarray), y (ndarray in {0,1})
    """
    phishing = fetch_ucirepo(id=327)  # Phishing Websites
    X = phishing.data.features.copy()
    y_df = phishing.data.targets.copy()

    # map target to {0,1}
    y_vals = np.ravel(y_df.values) if isinstance(y_df, pd.DataFrame) else np.asarray(y_df)
    uniq = np.unique(y_vals)
    if len(uniq) != 2:
        raise ValueError(f"Expected binary target, got classes: {uniq}")
    mapping = {uniq[0]: 0, uniq[1]: 1}
    y = np.vectorize(mapping.get)(y_vals)

    # correlation prune (your helper)
    X_reduced, removed = remove_highly_correlated_features_pd(X, threshold)
    print("Removed due to correlation:", removed)

    # ensure ndarray
    if isinstance(X_reduced, pd.DataFrame):
        X_arr = X_reduced.to_numpy()
    else:
        X_arr = np.asarray(X_reduced)

    # scale
    if scale == "standard":
        X_scaled = StandardScaler().fit_transform(X_arr)
    elif scale == "minmax":
        X_min = X_arr.min(axis=0, keepdims=True)
        X_max = X_arr.max(axis=0, keepdims=True)
        denom = np.where((X_max - X_min) == 0, 1.0, (X_max - X_min))
        X_scaled = (X_arr - X_min) / denom
    else:
        X_scaled = X_arr

    print("Shape of X:", X_scaled.shape)
    print("Shape of y:", y.shape)
    return X_scaled, y


import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def get_tunadromd_XY(csv_path="/content/TUANDROMD.csv", threshold=0.80, scale="standard"):
    """
    Load TUNADROMD dataset from a CSV file (binary malware vs goodware).
    Prune features by correlation threshold, scale, and return arrays.
    """
    # Load CSV
    df = pd.read_csv(csv_path)

    # Assume last column is target
    X_df = df.iloc[:, :-1]
    y_df = df.iloc[:, -1]

    # Drop rows with missing target
    mask = ~pd.isna(y_df)
    X_df = X_df[mask]
    y_df = y_df[mask]

    # Convert target to numpy array
    y_vals = np.ravel(y_df.values)
    uniq = np.unique(y_vals)

    if len(uniq) != 2:
        raise ValueError(f"Expected binary target but got classes: {uniq}")

    mapping = {uniq[0]: 0, uniq[1]: 1}
    y = np.vectorize(mapping.get)(y_vals)

    # Correlation prune
    X_reduced, removed = remove_highly_correlated_features_pd(X_df, threshold)
    print("Removed due to correlation:", removed)

    # Ensure ndarray
    X_arr = X_reduced.to_numpy() if isinstance(X_reduced, pd.DataFrame) else np.asarray(X_reduced)

    # Scaling
    if scale == "standard":
        X_scaled = StandardScaler().fit_transform(X_arr)
    elif scale == "minmax":
        X_min = X_arr.min(axis=0, keepdims=True)
        X_max = X_arr.max(axis=0, keepdims=True)
        denom = np.where((X_max - X_min) == 0, 1.0, (X_max - X_min))
        X_scaled = (X_arr - X_min) / denom
    else:
        X_scaled = X_arr

    print("Shape of X:", X_scaled.shape)
    print("Shape of y:", y.shape)
    return X_scaled, y


def get_XY(dataset_name):
  if dataset_name == 'tunadromd':
    X,y = get_tunadromd_XY()
  elif dataset_name == 'phishing':
    X,y = get_phishing_XY()
  elif dataset_name == 'breastcancer':
    X,y = get_breast_cancer_XY()
  else:
    X, y = None, None
    print("Unknown Dataset Name")

  return X,y

In [ ]:
### test the Bayesian LR implementation aainst sklearn frequestist implementation
from sklearn.linear_model import LogisticRegression

# pick your prior std (σ); e.g., 1.0
prior_std = 1
# Frequentist (liblinear) — set C = σ²
freq_logit = LogisticRegression(
    penalty='l2', solver='liblinear',
    C=prior_std**2,              # <-- C = σ²
    fit_intercept=True, max_iter=5000, tol=1e-8
)

# Bayesian — use prior_std and DO NOT penalize the intercept
bayes_logit = BayesianLogisticPyMC(
    prior_std=prior_std,         # <-- σ
    fit_intercept=True,
    penalize_intercept=True,    # use Flat/very-wide prior for intercept
    advi_iters=40000,
    draws=5000,
    random_state=0,
)

num_runs = 30
dataset_names = ['phishing', 'tunadromd', 'breastcancer']

results_dict = {}

for dataset_name in dataset_names:
  print("Dataset: ", dataset_name)
  X, y = get_XY(dataset_name)

  corr_list = []
  error_list = []
  for num_run in np.arange(0, num_runs,1):

    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.8, stratify=y)

    # Ensure: same StandardScaler on X, intercept unpenalized in Bayes
    freq_logit.fit(X_tr, y_tr)
    bayes_logit.fit(X_tr, y_tr)

    w_freq  = freq_logit.coef_.ravel()
    w_bayes = bayes_logit.coef_.ravel()

    corr_list.append(np.corrcoef(w_freq, w_bayes)[0,1])
    error_list.append(np.linalg.norm(w_freq - w_bayes)/(np.linalg.norm(w_freq) + 1e-12))
    print("corr =", np.corrcoef(w_freq, w_bayes)[0,1])
    print("rel err =", np.linalg.norm(w_freq - w_bayes)/(np.linalg.norm(w_freq) + 1e-12))

  results_dict[dataset_name] = {'corr': np.array(corr_list), 'rel_err': np.array(error_list)}

Dataset:  phishing
Removed due to correlation: ['shortining_service', 'favicon']
Shape of X: (11055, 28)
Shape of y: (11055,)
corr = 0.9998063695711034
rel err = 0.056674998992691746
corr = 0.9997965669908284
rel err = 0.05732944885193848
corr = 0.999816533286539
rel err = 0.05730218645370612
corr = 0.9998090707167628
rel err = 0.056714706250924486
corr = 0.9997802131870787
rel err = 0.05292989043093497
corr = 0.9978039917952334
rel err = 0.07798471160872661
corr = 0.9997323002079742
rel err = 0.06338542290460981
corr = 0.999827484202003
rel err = 0.05418422640650362
corr = 0.9997440314471384
rel err = 0.0593227222279593
corr = 0.9997757313457034
rel err = 0.052606302475916814
corr = 0.9998299629124506
rel err = 0.053259848864482984
corr = 0.9997064726248709
rel err = 0.06530810267017687
corr = 0.7969763627707878
rel err = 0.7474423844136558


In [ ]:
###optimizing slack

# pip install optuna

import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import f1_score

# ---------- helpers (same as we used before) ----------
def _sigmoid(z): return 1.0 / (1.0 + np.exp(-z))

def _predict_proba_safe(clf, X):
    if hasattr(clf, "predict_proba"):
        proba = clf.predict_proba(X)
        if proba.ndim == 1 or proba.shape[1] == 1:
            p1 = np.ravel(proba); proba = np.c_[1 - p1, p1]
        return proba
    elif hasattr(clf, "decision_function"):
        z = clf.decision_function(X)
        z = np.ravel(z) if getattr(z, "ndim", 1) > 1 else z
        p1 = _sigmoid(z)
        return np.c_[1 - p1, p1]
    else:
        yhat = clf.predict(X).astype(int)
        proba = np.zeros((len(yhat), 2))
        proba[np.arange(len(yhat)), yhat] = 1.0
        return proba

def _strat_boot_idx(y, rng):
    classes, counts = np.unique(y, return_counts=True)
    if len(classes) < 2: return None
    idxs = np.concatenate([rng.choice(np.where(y==c)[0], size=cnt, replace=True)
                           for c, cnt in zip(classes, counts)])
    rng.shuffle(idxs)
    return idxs

def _bootstrap_coef_matrix(X, y, base_estimator, n_bootstraps=200, max_attempts_factor=5, random_state=123):
    rng = np.random.RandomState(random_state)
    coefs, attempts = [], 0
    target = n_bootstraps
    max_attempts = max_attempts_factor * n_bootstraps
    while len(coefs) < target and attempts < max_attempts:
        attempts += 1
        idx = _strat_boot_idx(y, rng)
        if idx is None: break
        Xb, yb = X[idx], y[idx]
        if len(np.unique(yb)) < 2:
            continue
        try:
            clf = clone(base_estimator).fit(Xb, yb)
            beta = getattr(clf, "coef_", None)
            if beta is None: continue
            beta = np.ravel(beta)
            if not np.all(np.isfinite(beta)): continue
            coefs.append(beta)
        except Exception:
            continue
    return np.array(coefs)  # (n_success, p)

def _freq_sign_entropy_on(X, y, evaluator, n_bootstraps=200, random_state=123):
    coef_mat = _bootstrap_coef_matrix(X, y, evaluator, n_bootstraps=n_bootstraps, random_state=random_state)
    if coef_mat.shape[0] < 10:
        return np.full(X.shape[1], np.nan)
    return calculate_entropies(coef_mat)  # <- your KDE-based per-feature entropy

# --------- Multi-objective BO: minimize (-F1, Entropy) ----------
def pareto_optimize_baycofe_slack(
    X_train, y_train, *,
    bayes_selector_estimator,   # your BayesianLogisticPyMC (unfitted)
    evaluator,                  # common downstream evaluator (e.g., freq_logit)
    n_trials=30,
    cv_splits=5,
    slack_low=1e-4,
    slack_high=1e-1,
    bootstrap_for_entropy=150,  # small in CV for speed
    random_state=123
):
    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=random_state)

    def objective(trial):
        slack = trial.suggest_float("slack", slack_low, slack_high, log=True)
        f1s, ents = [], []

        for tr_idx, va_idx in skf.split(X_train, y_train):
            X_tr, X_va = X_train[tr_idx], X_train[va_idx]
            y_tr, y_va = y_train[tr_idx], y_train[va_idx]

            # 1) select with BayCOFE (this uses the Bayesian estimator internally)
            sel_idx, _ = baycofe_select_features_cls(
                X_tr, y_tr,
                estimator=bayes_selector_estimator,
                slack=slack, num_iter=10, tolerance_limit=3
            )
            sel_idx = np.asarray(sel_idx, dtype=int)
            if sel_idx.size == 0:
                # degenerate fold; pessimistic values
                f1s.append(0.0)
                ents.append(1.0)
                continue

            # 2) fit the same evaluator on TRAIN[selected]
            eval_clf = clone(evaluator).fit(X_tr[:, sel_idx], y_tr)

            # 3) score on VAL[selected]
            proba = _predict_proba_safe(eval_clf, X_va[:, sel_idx])
            y_pred = (proba[:, 1] >= 0.5).astype(int)
            f1s.append(f1_score(y_va, y_pred, average="macro"))

            # 4) entropy on VAL[selected] via frequentist bootstrap (same evaluator)
            ent = _freq_sign_entropy_on(
                X_va[:, sel_idx], y_va, evaluator=eval_clf,
                n_bootstraps=bootstrap_for_entropy, random_state=trial.number + 17
            )
            ents.append(np.nanmean(ent))

        mean_f1 = float(np.mean(f1s))
        mean_ent = float(np.nanmean(ents))

        # store for convenience
        trial.set_user_attr("mean_f1", mean_f1)
        trial.set_user_attr("mean_entropy", mean_ent)

        # Multi-objective: minimize (-F1, Entropy)  <=>  maximize F1, minimize Entropy
        return -mean_f1, mean_ent

    study = optuna.create_study(
        directions=["minimize", "minimize"],
        sampler=optuna.samplers.TPESampler(seed=random_state)
    )
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

    # Extract Pareto set
    pareto = study.best_trials  # list of Pareto-optimal trials

    # ---- Knee selection on Pareto front ----
    # We want the point closest to the "ideal" (max F1, min Entropy).
    f1_vals = np.array([-t.values[0] for t in pareto])  # back to +F1
    ent_vals = np.array([t.values[1] for t in pareto])

    # normalize each axis to [0,1] over the Pareto set
    eps = 1e-12
    f1_min, f1_max = f1_vals.min(), f1_vals.max()
    ent_min, ent_max = ent_vals.min(), ent_vals.max()

    f1_norm = (f1_max - f1_vals) / (max(eps, f1_max - f1_min))  # 0 at ideal (max F1)
    ent_norm = (ent_vals - ent_min) / (max(eps, ent_max - ent_min))  # 0 at ideal (min Ent)

    dist = np.sqrt(f1_norm**2 + ent_norm**2)
    knee_idx = int(np.argmin(dist))
    knee_trial = pareto[knee_idx]
    knee_slack = knee_trial.params["slack"]

    return knee_slack, study, pareto

In [ ]:
# Final Driver
from sklearn.datasets import make_classification
import pickle

# --- init once ---
save_dir = '/content/drive/MyDrive/PhD/Results/bay_cofe_new/logistic_regression/'
methods = ["BayCOFE", "CoFE", "RFE", "SFS", "SBS", "Bidirectional"]
results = {m: {"accuracy": [], "entropy": [], "slack": [], "selected": []} for m in methods}

def _add(method, sel_idx, metrics, slack_val=None):
    # make everything JSON-friendly (no numpy types)
    results[method]["accuracy"].append(float(metrics.get("accuracy", np.nan)))
    results[method]["entropy"].append(
        float(metrics.get("mean_entropy")) if metrics.get("mean_entropy") is not None else None
    )
    results[method]["slack"].append(None if slack_val is None else float(slack_val))
    results[method]["selected"].append(np.asarray(sel_idx, dtype=int).tolist())


num_runs = 20

for i in np.arange(0,num_runs):
    X, y = get_tunadromd_XY() #get_phishing_XY() # get_breast_cancer_XY() #
    X = StandardScaler().fit_transform(X)
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.5, stratify=y) #random_state=42,


    # Estimators
    prior_std = 3.0
    bayes_logit = BayesianLogisticPyMC(
        prior_std=prior_std, fit_intercept=True, penalize_intercept=False,
        advi_iters=30000, draws=3000, random_state=0
    )
    freq_logit = LogisticRegression(penalty='l2', solver='liblinear', C=prior_std, max_iter=2000, tol=1e-6)

    #knee_slack = 0.012285916427548801


    # Multi-objective BO for slack (TRAIN only)
    knee_slack, study, pareto = pareto_optimize_baycofe_slack(
        X_tr, y_tr,
        bayes_selector_estimator=bayes_logit,
        evaluator=freq_logit,
        n_trials=8,           # bump up if you have time
        cv_splits=2,
        slack_low=1e-4, slack_high=1e-1,
        bootstrap_for_entropy=150,
        random_state=123
    )

    print("Knee slack:", knee_slack)

    # Final train on full TRAIN with knee slack, evaluate on TEST
    sel_idx, metrics = evaluate_feature_selection_method_cls(
        baycofe_select_features_cls,
        X_tr, y_tr, X_te, y_te,
        selector_params={'estimator': bayes_logit, 'evaluator': freq_logit, 'slack': knee_slack}
    )
    k = len(sel_idx)
    print("BayCoFE Selected:", sel_idx)
    print("Test Acc:", metrics["accuracy"], "LogLoss:", metrics["log_loss"], "Mean Entropy:", metrics["mean_entropy"])
    _add("BayCOFE", sel_idx, metrics, slack_val=knee_slack)


    sel_idx, metrics = evaluate_feature_selection_method_cls(
        sefe_select_features,  # now Bayesian version
        X_tr, y_tr, X_te, y_te,
        selector_params={'estimator': freq_logit, 'evaluator': freq_logit,'slack': knee_slack})
    print("CoFE Selected:", sel_idx)
    print("COFE Acc:", metrics["accuracy"], "LogLoss:", metrics["log_loss"], "entropy:", metrics['mean_entropy'])
    _add("CoFE", sel_idx, metrics, slack_val=knee_slack)


    sel_idx, metrics = evaluate_feature_selection_method_cls(
        rfe_select_features,
        X_tr, y_tr, X_te, y_te,
        selector_params={'estimator': freq_logit, 'evaluator': freq_logit, 'min_k':k})
    print("RFE Selected:", sel_idx)
    print("RFE Acc:", metrics["accuracy"], "LogLoss:", metrics["log_loss"], "entropy:", metrics['mean_entropy'])
    _add("RFE", sel_idx, metrics, slack_val=knee_slack)


    sel_idx, metrics = evaluate_feature_selection_method_cls(
    sfs_select_features,
    X_tr, y_tr, X_te, y_te,
    selector_params={'estimator': freq_logit, 'evaluator': freq_logit, 'min_k':k})
    print("SFS Selected:", sel_idx)
    print("SFS Acc:", metrics["accuracy"], "LogLoss:", metrics["log_loss"], "entropy:", metrics['mean_entropy'])
    _add("SFS", sel_idx, metrics, slack_val=knee_slack)


    sel_idx, metrics = evaluate_feature_selection_method_cls(
        sbs_select_features,
        X_tr, y_tr, X_te, y_te,
        selector_params={'estimator': freq_logit, 'evaluator': freq_logit, 'min_k':k})
    print("SBS Selected:", sel_idx)
    print("SBS Acc:", metrics["accuracy"], "LogLoss:", metrics["log_loss"], "entropy:", metrics['mean_entropy'])
    _add("SBS", sel_idx, metrics, slack_val=knee_slack)


    sel_idx, metrics = evaluate_feature_selection_method_cls(
    bidir_select_features,
    X_tr, y_tr, X_te, y_te,
    selector_params={'estimator': freq_logit, 'evaluator': freq_logit, 'min_k':k})
    print("Bidirectional Selected:", sel_idx)
    print("Bidirectional Acc:", metrics["accuracy"], "LogLoss:", metrics["log_loss"], "entropy:", metrics['mean_entropy'])
    _add("Bidirectional", sel_idx, metrics, slack_val=knee_slack)


# save
with open(save_dir + "TUANDROMD_dataset.pkl", "wb") as f:
    pickle.dump(results, f, protocol=pickle.HIGHEST_PROTOCOL)

[I 2025-10-01 05:10:05,632] A new study created in memory with name: no-name-64abada1-737e-4004-aef8-0ccbbe17928c


Removed due to correlation: ['ACCESS_CHECKIN_PROPERTIES', 'ACCESS_COARSE_LOCATION', 'ACCESS_LOCATION_EXTRA_COMMANDS', 'ACCESS_MOCK_LOCATION', 'ACCESS_SURFACE_FLINGER', 'ACCESS_WIFI_STATE', 'ACCOUNT_MANAGER', 'ADD_VOICEMAIL', 'AUTHENTICATE_ACCOUNTS', 'BIND_ACCESSIBILITY_SERVICE', 'BIND_APPWIDGET', 'BIND_CARRIER_MESSAGING_SERVICE', 'BIND_DREAM_SERVICE', 'BIND_INPUT_METHOD', 'BIND_NFC_SERVICE', 'BIND_0TIFICATION_LISTENER_SERVICE', 'BIND_PRINT_SERVICE', 'BIND_REMOTEVIEWS', 'BIND_TEXT_SERVICE', 'BIND_TV_INPUT', 'BIND_VOICE_INTERACTION', 'BIND_VPN_SERVICE', 'BIND_WALLPAPER', 'BLUETOOTH', 'BLUETOOTH_PRIVILEGED', 'BODY_SENSORS', 'BRICK', 'BROADCAST_PACKAGE_REMOVED', 'BROADCAST_SMS', 'BROADCAST_STICKY', 'BROADCAST_WAP_PUSH', 'CALL_PRIVILEGED', 'CAMERA', 'CAPTURE_AUDIO_OUTPUT', 'CAPTURE_SECURE_VIDEO_OUTPUT', 'CAPTURE_VIDEO_OUTPUT', 'CHANGE_COMPONENT_ENABLED_STATE', 'CHANGE_CONFIGURATION', 'CHANGE_WIFI_MULTICAST_STATE', 'CLEAR_APP_CACHE', 'CLEAR_APP_USER_DATA', 'CONTROL_LOCATION_UPDATES', 'DELETE

[I 2025-10-01 05:10:53,640] Trial 0 finished with values: [-0.9522885555006535, 0.04167368983339427] and parameters: {'slack': 0.012285916427548801}.
[I 2025-10-01 05:11:43,031] Trial 1 finished with values: [-0.895916945700338, 0.0] and parameters: {'slack': 0.0007218018732952586}.
[I 2025-10-01 05:12:34,972] Trial 2 finished with values: [-0.8726808841843992, 0.0] and parameters: {'slack': 0.0004792414358623592}.
[I 2025-10-01 05:13:20,964] Trial 3 finished with values: [-0.9526737378520707, 0.0033507024124500226] and parameters: {'slack': 0.004507588967315677}.
[I 2025-10-01 05:14:06,883] Trial 4 finished with values: [-0.9509762384675695, 0.10520319831145826] and parameters: {'slack': 0.014401472828461642}.
[I 2025-10-01 05:14:56,079] Trial 5 finished with values: [-0.9257716536055614, 0.0] and parameters: {'slack': 0.0018591711878786364}.
[I 2025-10-01 05:15:52,805] Trial 6 finished with values: [-0.9569227705239268, 0.12631662235481428] and parameters: {'slack': 0.087557347250565

Knee slack: 0.004507588967315677
BayCoFE Selected: [ 61  64  66  76  80  96  98  99 107]
Test Acc: 0.9744623655913979 LogLoss: 0.08436296678243352 Mean Entropy: 0.0
CoFE Selected: [  0   1   4   6   7   8  10  11  12  13  14  15  17  19  21  23  26  27
  29  30  31  32  35  36  37  39  41  42  43  46  47  48  49  50  51  52
  53  54  56  57  59  60  61  62  63  64  65  66  67  69  71  72  73  75
  76  79  80  82  83  85  86  89  90  96  99 107 113]
COFE Acc: 0.9762544802867383 LogLoss: 0.0824840414466717 entropy: 0.008402347798825732
RFE Selected: [  5  16  22  24  40  55  58  61  64  66  74  76  80  81  87  91  96  98
  99 101 103 107 113 118]
RFE Acc: 0.9843189964157706 LogLoss: 0.050735343885552106 entropy: 0.09884787088450502
SFS Selected: [  0   1   2   4  24  40  64  66  80  81  99 101 105 111]
SFS Acc: 0.975358422939068 LogLoss: 0.07770411193890783 entropy: 0.03110555620717534
SBS Selected: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21

[I 2025-10-01 05:25:28,878] A new study created in memory with name: no-name-879527f9-4e4f-46c6-89b4-a2c74540d800


...

Knee slack: 0.014401472828461642
BayCoFE Selected: [ 16  58  61  64  66  76  80  87  96  98  99 107 113]
Test Acc: 0.9726702508960573 LogLoss: 0.06858617481893467 Mean Entropy: 0.030080375816256374
CoFE Selected: [  0   1   4   6   7   8  10  11  12  13  14  15  16  19  21  23  24  26
  27  29  30  31  32  35  36  37  39  40  41  42  43  46  47  48  49  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  69
  71  72  73  75  76  79  80  82  83  85  86  87  88  89  90  96  99 107
 111 113]
COFE Acc: 0.9825268817204301 LogLoss: 0.051332875230770694 entropy: 0.012041848970759841
RFE Selected: [  5  16  22  24  25  28  33  34  38  40  44  45  55  58  61  64  66  68
  70  74  76  78  79  80  81  87  91  93  94  95  96  98  99 100 102 103
 104 107 109 111 112 113 115]
RFE Acc: 0.9829749103942652 LogLoss: 0.05172631332478502 entropy: 0.2289040003413211
SFS Selected: [  0   1   2   3   4   6   7   8   9  10  11  12  13  14  15  17  18  19
  21  23  25  26  27  28  29  31  

[I 2025-10-01 11:14:43,750] A new study created in memory with name: no-name-4280cf5f-9748-4592-85d8-db81170d7350


Removed due to correlation: ['ACCESS_CHECKIN_PROPERTIES', 'ACCESS_COARSE_LOCATION', 'ACCESS_LOCATION_EXTRA_COMMANDS', 'ACCESS_MOCK_LOCATION', 'ACCESS_SURFACE_FLINGER', 'ACCESS_WIFI_STATE', 'ACCOUNT_MANAGER', 'ADD_VOICEMAIL', 'AUTHENTICATE_ACCOUNTS', 'BIND_ACCESSIBILITY_SERVICE', 'BIND_APPWIDGET', 'BIND_CARRIER_MESSAGING_SERVICE', 'BIND_DREAM_SERVICE', 'BIND_INPUT_METHOD', 'BIND_NFC_SERVICE', 'BIND_0TIFICATION_LISTENER_SERVICE', 'BIND_PRINT_SERVICE', 'BIND_REMOTEVIEWS', 'BIND_TEXT_SERVICE', 'BIND_TV_INPUT', 'BIND_VOICE_INTERACTION', 'BIND_VPN_SERVICE', 'BIND_WALLPAPER', 'BLUETOOTH', 'BLUETOOTH_PRIVILEGED', 'BODY_SENSORS', 'BRICK', 'BROADCAST_PACKAGE_REMOVED', 'BROADCAST_SMS', 'BROADCAST_STICKY', 'BROADCAST_WAP_PUSH', 'CALL_PRIVILEGED', 'CAMERA', 'CAPTURE_AUDIO_OUTPUT', 'CAPTURE_SECURE_VIDEO_OUTPUT', 'CAPTURE_VIDEO_OUTPUT', 'CHANGE_COMPONENT_ENABLED_STATE', 'CHANGE_CONFIGURATION', 'CHANGE_WIFI_MULTICAST_STATE', 'CLEAR_APP_CACHE', 'CLEAR_APP_USER_DATA', 'CONTROL_LOCATION_UPDATES', 'DELETE

[I 2025-10-01 11:15:36,656] Trial 0 finished with values: [-0.9448736265392518, 0.06246385623517342] and parameters: {'slack': 0.012285916427548801}.
[I 2025-10-01 11:16:26,055] Trial 1 finished with values: [-0.8795713376028724, 0.0] and parameters: {'slack': 0.0007218018732952586}.
[I 2025-10-01 11:17:15,899] Trial 2 finished with values: [-0.8795713376028724, 0.0] and parameters: {'slack': 0.0004792414358623592}.
[I 2025-10-01 11:18:02,371] Trial 3 finished with values: [-0.9379404484998711, 0.06205563334238837] and parameters: {'slack': 0.004507588967315677}.
[I 2025-10-01 11:18:52,900] Trial 4 finished with values: [-0.9448736265392518, 0.06173128142319697] and parameters: {'slack': 0.014401472828461642}.
[I 2025-10-01 11:19:43,531] Trial 5 finished with values: [-0.9064071746182831, 0.0] and parameters: {'slack': 0.0018591711878786364}.
[I 2025-10-01 11:20:36,342] Trial 6 finished with values: [-0.9623684932821703, 0.04084045420799486] and parameters: {'slack': 0.0875573472505650

Knee slack: 0.0018591711878786364
BayCoFE Selected: [ 61  64  66  76  80  96  98 107 113]
Test Acc: 0.9735663082437276 LogLoss: 0.091733270841204 Mean Entropy: 0.001197133211296129
CoFE Selected: [  0   1   4   6   7   8  10  11  12  13  14  15  16  17  19  21  23  26
  27  29  30  31  32  35  36  37  39  41  42  43  46  47  48  49  50  51
  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  69  71
  72  73  75  76  79  80  82  83  85  86  87  88  89  90  96  98  99 107
 113]
COFE Acc: 0.978494623655914 LogLoss: 0.0673437394389538 entropy: 0.016720491497769305
RFE Selected: [  2  16  22  24  33  34  40  55  61  64  66  70  74  76  79  80  84  87
  91  96  98  99 107 109 111 112 113 116 117]
RFE Acc: 0.9834229390681004 LogLoss: 0.05023398096771209 entropy: 0.16878009754149598
SFS Selected: [  0   1  20  64  74  95  99 108 113]
SFS Acc: 0.9708781362007168 LogLoss: 0.11635494882850347 entropy: 0.004201902648732676
SBS Selected: [  0   1   2   3   4   5   6   7   8   9  10  11

In [10]:
### Relevant features

# Function to compute Intersection-over-Union (IoU) between two sets.
def iou(set1, set2):
    if len(set1) == 0 and len(set2) == 0:
        return 1.0
    intersection = np.intersect1d(set1, set2)
    union = np.union1d(set1, set2)
    return len(intersection) / len(union)

# Function to compute a pairwise IoU matrix given a list of feature sets.
def compute_pairwise_iou(feature_sets):
    num = len(feature_sets)
    iou_matrix = np.zeros((num, num))
    for i in range(num):
        for j in range(num):
            iou_matrix[i, j] = iou(feature_sets[i], feature_sets[j])
    return iou_matrix


def generate_synthetic_data_logistic(n_true=10, m_noise=20, num_samples=1500,
                                     noise_type='gaussian', noise_level=0.1):
    # Generate true features
    X_true = np.random.randn(num_samples, n_true)

    # True coefficients
    beta = np.random.randn(n_true)

    # Adjust noise levels
    if noise_type == 'gaussian':
        noise = np.random.normal(0, noise_level, X_true.shape)
    elif noise_type == 'uniform':
        noise = np.random.uniform(-np.sqrt(3) * noise_level,
                                  np.sqrt(3) * noise_level, X_true.shape)
    elif noise_type == 'laplace':
        noise = np.random.laplace(0, noise_level / np.sqrt(2), X_true.shape)
    elif noise_type == 'heteroscedastic':
        noise = noise_level * (1 + np.abs(X_true)) * np.random.randn(*X_true.shape)
    elif noise_type == 'correlated':
        noise = noise_level * np.sin(np.linspace(0, np.pi, num_samples)).reshape(-1, 1)
    else:
        raise ValueError("Invalid noise type!")

    # Add noise to features
    X_true_noisy = X_true + noise

    # Logistic model: generate probabilities
    logits = X_true @ beta
    probs = 1 / (1 + np.exp(-logits))   # sigmoid

    # Sample binary outcomes
    y = np.random.binomial(1, probs)

    # Artificial noise features
    X_noise = np.random.randn(num_samples, m_noise)

    # Concatenate true + noise features
    X = np.hstack((X_true_noisy, X_noise))

    return X, y, np.arange(n_true)


def generate_synthetic_data(n_true=10, m_noise=20, num_samples=1500, noise_type='gaussian', noise_level=0.1):
    #np.random.seed(42)  # Reproducibility

    # Generate true features affecting y
    X_true = np.random.randn(num_samples, n_true)

    # Generate true coefficients
    beta = np.random.randn(n_true)

    # Adjust noise levels for fair comparison
    if noise_type == 'gaussian':
        noise = np.random.normal(0, noise_level, X_true.shape)  # Standard deviation = noise_level
    elif noise_type == 'uniform':
        noise = np.random.uniform(-np.sqrt(3) * noise_level, np.sqrt(3) * noise_level, X_true.shape)
        # Scaled so variance matches Gaussian: Var(U(a,b)) = (b-a)^2 / 12
    elif noise_type == 'laplace':
        noise = np.random.laplace(0, noise_level / np.sqrt(2), X_true.shape)
        # Scale adjusted so variance matches Gaussian: Var(Laplace(0, b)) = 2b^2
    elif noise_type == 'heteroscedastic':
        noise = noise_level * (1 + np.abs(X_true)) * np.random.randn(*X_true.shape)
        # Noise increases with value
    elif noise_type == 'correlated':
        noise = noise_level * np.sin(np.linspace(0, np.pi, num_samples)).reshape(-1, 1)
        # Sinusoidal correlated noise
    else:
        raise ValueError("Invalid noise type!")

    # Add noise to true features
    X_true_noisy = X_true + noise

    # Generate target variable y with additional Gaussian observation noise
    #y = X_true_noisy @ beta + np.random.normal(0, noise_level, num_samples) #0

    y = X_true @ beta + np.random.normal(0, noise_level, num_samples) #1

    # Artificial noisy features (completely random)
    X_noise = np.random.randn(num_samples, m_noise)

    # Concatenate true and noisy features
    X = np.hstack((X_true_noisy, X_noise))

    return X, y, np.arange(n_true)


# Function to evaluate feature selection
def evaluate_feature_selection_old(method_name, selected_features, true_features, total_features):
    selected_features = set(selected_features)
    true_features = set(true_features)

    # Compute IoU
    intersection = len(selected_features & true_features)
    union = len(selected_features | true_features)
    iou = intersection / union if union != 0 else 0

    # Compute False Positive Rate (FPR)
    false_positives = len(selected_features - true_features) / (total_features - len(true_features))

    # Compute False Negative Rate (FNR)
    false_negatives = len(true_features - selected_features) / len(true_features)

    print(f"{method_name}: IoU={iou:.3f}, FPR={false_positives:.3f}, FNR={false_negatives:.3f}")

    return iou, false_positives, false_negatives

def evaluate_feature_selection(method_name, selected_features, true_features, total_features):
    selected_features = set(selected_features)
    true_features = set(true_features)

    # --- Counts ---
    TP = len(selected_features & true_features)  # correctly selected
    FP = len(selected_features - true_features)  # irrelevant selected
    FN = len(true_features - selected_features)  # missed relevant
    TN = total_features - (TP + FP + FN)         # correctly rejected

    # --- IoU ---
    union = TP + FP + FN
    iou = TP / union if union != 0 else 0

    # --- FPR, FNR ---
    false_positives = FP / (FP + TN) if (FP + TN) > 0 else 0
    false_negatives = FN / (FN + TP) if (FN + TP) > 0 else 0

    # --- Precision, Recall, F1 ---
    precision_pos = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall_pos    = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_pos = 2 * precision_pos * recall_pos / (precision_pos + recall_pos) if (precision_pos + recall_pos) > 0 else 0

    precision_neg = TN / (TN + FN) if (TN + FN) > 0 else 0
    recall_neg    = TN / (TN + FP) if (TN + FP) > 0 else 0
    f1_neg = 2 * precision_neg * recall_neg / (precision_neg + recall_neg) if (precision_neg + recall_neg) > 0 else 0

    macro_f1 = (f1_pos + f1_neg) / 2

    print(f"{method_name}: IoU={iou:.3f}, FPR={false_positives:.3f}, FNR={false_negatives:.3f}, f1_pos={f1_pos:.3f}")

    return iou, false_positives, false_negatives, f1_pos

In [ ]:
from sklearn.model_selection import GridSearchCV
from mlxtend.feature_selection import SequentialFeatureSelector


def rfe_select_features(X_train, y_train, **kwargs):
    estimator = kwargs['estimator']
    max_k = X_train.shape[1]  # Maximum number of features

    param_grid = {'n_features_to_select': range(1, max_k + 1)}  # Define range of k values
    selector = RFE(estimator=estimator)
    grid_search = GridSearchCV(selector, param_grid, cv=5)  # 5-fold cross-validation
    grid_search.fit(X_train, y_train)  # X_train and y_train are your training data

    best_k = grid_search.best_params_['n_features_to_select']
    selected_features = [i for i, selected in enumerate(grid_search.best_estimator_.support_) if selected]

    return selected_features, None


def sfs_select_features(X_train, y_train, **kwargs):
    estimator = kwargs['estimator']
    n_features_to_select = kwargs.get('n_features_to_select', X_train.shape[1])

    selector = SequentialFeatureSelector(estimator=estimator, k_features=(1, X_train.shape[1]), cv=5, forward=True,
                                         floating=False)
    selector.fit(X_train, y_train)
    return selector.k_feature_idx_, None


def sbs_select_features(X_train, y_train, **kwargs):
    estimator = kwargs['estimator']
    n_features_to_select = kwargs.get('n_features_to_select', X_train.shape[1])
    selector = SequentialFeatureSelector(estimator=estimator, k_features=(1, X_train.shape[1]), cv=5, forward=False,
                                         floating=False)
    selector.fit(X_train, y_train)
    return selector.k_feature_idx_, None


def bidir_select_features(X_train, y_train, **kwargs):
    estimator = kwargs['estimator']
    n_features_to_select = kwargs.get('n_features_to_select', X_train.shape[1])
    selector = SequentialFeatureSelector(estimator=estimator, k_features=(1, X_train.shape[1]), cv=5, forward=True,
                                         floating=True)
    selector.fit(X_train, y_train)
    return selector.k_feature_idx_, None

In [ ]:
### driver for relevant features

# Run the evaluation
import pickle
import numpy as np
from sklearn.linear_model import BayesianRidge
import random
from sklearn.preprocessing import StandardScaler


if __name__ == '__main__':
    # Number of runs (independent datasets)
    num_runs = 20
    prior_std = 3


    # Define noise levels (sigma values)
    sigma_kernel_values = [0.001, 0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9, 2.1] #[1.7, 1.9, 2.1] #
    noise_type = "correlated"
    # Dictionary to store results
    results = {}
    selected_features_dict = {}

    for sigma in sigma_kernel_values:
        print(f"\nRunning feature selection for sigma={sigma}...")

        for run in np.arange(0, num_runs, step=1):
            print(f"Run {run+1}/{num_runs} for sigma={sigma}")

            bayes_logit = BayesianLogisticPyMC(
                prior_std=prior_std, fit_intercept=True, penalize_intercept=False,
                advi_iters=30000, draws=3000, random_state=0
            )
            freq_logit = LogisticRegression(penalty='l2', solver='liblinear', C=prior_std**2, fit_intercept=True, max_iter=5000, tol=1e-8)

            # Generate a fresh dataset for this run
            #datasets, true_features = generate_synthetic_data(sigma_kernel=[sigma])
            #X, y, _ = datasets[0]  # Extract the generated dataset
            num_true = np.random.randint(1, 11)
            num_noise = np.random.randint(10, 41)
            print(f"Num True:{num_true} Num Noisy: {num_noise}")
            X,y, true_features = generate_synthetic_data_logistic(n_true=num_true, m_noise=num_noise, noise_type=noise_type, noise_level=sigma)
            scaler = StandardScaler()
            X = scaler.fit_transform(X)
            # 1. CoFE (Frequentist)

            sel_features_cofe, _ = sefe_select_features(X, y, slack=0.0454, estimator=freq_logit)

            # Append results instead of overwriting
            results.setdefault(f"CoFE_sigma={sigma}", []).append(
                evaluate_feature_selection("CoFE", sel_features_cofe, true_features, X.shape[1])
            )
            selected_features_dict.setdefault(f"CoFE_sigma={sigma}", []).append({
                "selected": sel_features_cofe,
                "n_true": num_true,
                "m_noise": num_noise
            })


            # 2. Bay-CoFE
            sel_features_baycofe, _ = baycofe_select_features_cls(X, y, slack=0.0454, estimator=bayes_logit)
            results.setdefault(f"Bay-CoFE_sigma={sigma}", []).append(
                evaluate_feature_selection("Bay-CoFE", sel_features_baycofe, true_features, X.shape[1])
            )
            selected_features_dict.setdefault(f"Bay-CoFE_sigma={sigma}", []).append({
                  "selected": sel_features_baycofe,
                  "n_true": num_true,
                  "m_noise": num_noise
            })


            # 3. RFE (Recursive Feature Elimination)
            sel_features_rfe, _ = rfe_select_features(X, y, estimator=freq_logit)
            results.setdefault(f"RFE_sigma={sigma}", []).append(
                evaluate_feature_selection("RFE", sel_features_rfe, true_features, X.shape[1])
            )
            #selected_features_dict.setdefault(f"RFE_sigma={sigma}", []).append(sel_features_rfe)
            selected_features_dict.setdefault(f"RFE_sigma={sigma}", []).append({
                  "selected": sel_features_rfe,
                  "n_true": num_true,
                  "m_noise": num_noise
            })


            # 4. Sequential Forward Selection (SFS)
            sel_features_sfs, _ = sfs_select_features(X, y, estimator=freq_logit)
            results.setdefault(f"SFS_sigma={sigma}", []).append(
                evaluate_feature_selection("SFS", sel_features_sfs, true_features, X.shape[1])
            )
            #selected_features_dict.setdefault(f"SFS_sigma={sigma}", []).append(sel_features_sfs)
            selected_features_dict.setdefault(f"SFS_sigma={sigma}", []).append({
                  "selected": sel_features_sfs,
                  "n_true": num_true,
                  "m_noise": num_noise
            })


            # 5. Sequential Backward Selection (SBS)
            sel_features_sbs, _ = sbs_select_features(X, y, estimator=freq_logit)
            results.setdefault(f"SBS_sigma={sigma}", []).append(
                evaluate_feature_selection("SBS", sel_features_sbs, true_features, X.shape[1])
            )
            #selected_features_dict.setdefault(f"SBS_sigma={sigma}", []).append(sel_features_sbs)
            selected_features_dict.setdefault(f"SBS_sigma={sigma}", []).append({
                  "selected": sel_features_sbs,
                  "n_true": num_true,
                  "m_noise": num_noise
            })

            # 6. Bidirectional Feature Selection
            sel_features_bidir, _ = bidir_select_features(X, y, estimator=freq_logit)
            results.setdefault(f"Bidirectional_sigma={sigma}", []).append(
                evaluate_feature_selection("Bidirectional", sel_features_bidir, true_features, X.shape[1])
            )
            #selected_features_dict.setdefault(f"Bidirectional_sigma={sigma}", []).append(sel_features_bidir)
            selected_features_dict.setdefault(f"Bidirectional_sigma={sigma}", []).append({
                  "selected": sel_features_bidir,
                  "n_true": num_true,
                  "m_noise": num_noise
            })


Running feature selection for sigma=0.001...
Run 1/20 for sigma=0.001
Num True:1 Num Noisy: 39
CoFE: IoU=1.000, FPR=0.000, FNR=0.000, f1_pos=1.000
Bay-CoFE: IoU=1.000, FPR=0.000, FNR=0.000, f1_pos=1.000
RFE: IoU=0.167, FPR=0.128, FNR=0.000, f1_pos=0.286
SFS: IoU=0.067, FPR=0.359, FNR=0.000, f1_pos=0.125
SBS: IoU=0.067, FPR=0.359, FNR=0.000, f1_pos=0.125
Bidirectional: IoU=0.071, FPR=0.333, FNR=0.000, f1_pos=0.133
Run 2/20 for sigma=0.001
Num True:4 Num Noisy: 22
CoFE: IoU=1.000, FPR=0.000, FNR=0.000, f1_pos=1.000
Bay-CoFE: IoU=1.000, FPR=0.000, FNR=0.000, f1_pos=1.000
RFE: IoU=1.000, FPR=0.000, FNR=0.000, f1_pos=1.000
SFS: IoU=0.571, FPR=0.136, FNR=0.000, f1_pos=0.727
SBS: IoU=0.364, FPR=0.318, FNR=0.000, f1_pos=0.533
Bidirectional: IoU=0.364, FPR=0.318, FNR=0.000, f1_pos=0.533
Run 3/20 for sigma=0.001
Num True:2 Num Noisy: 22
CoFE: IoU=1.000, FPR=0.000, FNR=0.000, f1_pos=1.000
Bay-CoFE: IoU=1.000, FPR=0.000, FNR=0.000, f1_pos=1.000
RFE: IoU=0.400, FPR=0.136, FNR=0.000, f1_pos=0.571
S

In [ ]:
# Save results
pkl_filename = f"/content/drive/MyDrive/PhD/Results/bay_cofe_new/relevant_features/20_runs_feature_relevance_evaluation_logistic_{noise_type}_std_{prior_std}.pkl"
with open(pkl_filename, 'wb') as f:
  pickle.dump({"results": results, "sel_features": selected_features_dict}, f)

In [ ]:
import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import f1_score

# --- tiny helpers (reuse yours if already defined) ---
def _sigmoid(z): return 1.0/(1.0+np.exp(-z))
def _predict_proba_safe(clf, X):
    if hasattr(clf, "predict_proba"):
        proba = clf.predict_proba(X)
        if proba.ndim == 1 or proba.shape[1] == 1:
            p1 = np.ravel(proba); proba = np.c_[1-p1, p1]
        return proba
    elif hasattr(clf, "decision_function"):
        z = clf.decision_function(X)
        z = np.ravel(z) if getattr(z, "ndim", 1) > 1 else z
        p1 = _sigmoid(z)
        return np.c_[1-p1, p1]
    else:
        yhat = clf.predict(X).astype(int)
        proba = np.zeros((len(yhat), 2)); proba[np.arange(len(yhat)), yhat] = 1.0
        return proba

def _strat_boot_idx(y, rng):
    classes, counts = np.unique(y, return_counts=True)
    if len(classes) < 2: return None
    idxs = np.concatenate([rng.choice(np.where(y==c)[0], size=cnt, replace=True)
                           for c, cnt in zip(classes, counts)])
    rng.shuffle(idxs); return idxs

def _bootstrap_coef_matrix(X, y, base_estimator, n_bootstraps=150, random_state=123):
    rng = np.random.RandomState(random_state)
    coefs, attempts = [], 0
    max_attempts = 5 * n_bootstraps
    while len(coefs) < n_bootstraps and attempts < max_attempts:
        attempts += 1
        idx = _strat_boot_idx(y, rng)
        if idx is None: break
        Xb, yb = X[idx], y[idx]
        if len(np.unique(yb)) < 2: continue
        try:
            bs = clone(base_estimator).fit(Xb, yb)
            beta = getattr(bs, "coef_", None)
            if beta is None: continue
            beta = np.ravel(beta)
            if not np.all(np.isfinite(beta)): continue
            coefs.append(beta)
        except Exception:
            continue
    return np.array(coefs)

def _freq_sign_entropy_on(X, y, evaluator, n_bootstraps=150, random_state=123):
    B = _bootstrap_coef_matrix(X, y, evaluator, n_bootstraps=n_bootstraps, random_state=random_state)
    if B.shape[0] < 10:  # too few successes => return NaNs
        return np.full(X.shape[1], np.nan)
    return calculate_entropies(B)  # <-- your KDE-based per-feature entropy


# --- build the objective as a closure over your data/estimators ---
def make_objective(X_train, y_train, bayes_selector_estimator, evaluator,
                   cv_splits=5, slack_low=1e-4, slack_high=1e-1,
                   bootstrap_for_entropy=150, seed=123):
    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=seed)
    def objective(trial):
        slack = trial.suggest_float("slack", slack_low, slack_high, log=True)
        f1s, ents = [], []
        for tr_idx, va_idx in skf.split(X_train, y_train):
            X_tr, X_va = X_train[tr_idx], X_train[va_idx]
            y_tr, y_va = y_train[tr_idx], y_train[va_idx]

            # BayCOFE selection on this fold
            sel_idx, _ = baycofe_select_features_cls(
                X_tr, y_tr,
                estimator=bayes_selector_estimator,
                slack=slack, num_iter=10, tolerance_limit=3
            )
            sel_idx = np.asarray(sel_idx, dtype=int)
            if sel_idx.size == 0:
                f1s.append(0.0); ents.append(1.0); continue

            # Fit SAME evaluator and score
            ev = clone(evaluator).fit(X_tr[:, sel_idx], y_tr)
            proba = _predict_proba_safe(ev, X_va[:, sel_idx])
            y_pred = (proba[:,1] >= 0.5).astype(int)
            f1s.append(f1_score(y_va, y_pred, average="macro"))

            # Entropy on VAL subset (bootstrap evaluator)
            ent = _freq_sign_entropy_on(X_va[:, sel_idx], y_va, evaluator=ev,
                                        n_bootstraps=bootstrap_for_entropy,
                                        random_state=trial.number + 17)
            ents.append(np.nanmean(ent))

        mean_f1  = float(np.mean(f1s))
        mean_ent = float(np.nanmean(ents))
        trial.set_user_attr("mean_f1", mean_f1)
        trial.set_user_attr("mean_entropy", mean_ent)
        # Multi-objective: minimize (-F1, Entropy)
        return -mean_f1, mean_ent
    return objective


# --- optional early-stop callback for few trials ---
def make_plateau_callback(patience=4, f1_tol=0.002, ent_tol=0.01):
    state = {"best_f1": -np.inf, "best_ent": np.inf, "stale": 0}
    def cb(study, trial):
        f1  = trial.user_attrs.get("mean_f1")
        ent = trial.user_attrs.get("mean_entropy")
        improved = False
        if f1 is not None and f1 > state["best_f1"] + f1_tol:
            state["best_f1"] = f1; improved = True
        if ent is not None and ent < state["best_ent"] - ent_tol:
            state["best_ent"] = ent; improved = True
        state["stale"] = 0 if improved else state["stale"] + 1
        if state["stale"] >= patience:
            study.stop()
    return cb


sampler = optuna.samplers.TPESampler(seed=123, n_startup_trials=3)
objective = make_objective(
    X_tr, y_tr,
    bayes_selector_estimator=bayes_logit,
    evaluator=freq_logit,
    cv_splits=5,
    slack_low=1e-4, slack_high=1e-1,
    bootstrap_for_entropy=150,  # small for CV speed
    seed=123
)

study = optuna.create_study(directions=["minimize","minimize"], sampler=sampler)
study.optimize(objective, n_trials=12, callbacks=[ make_plateau_callback() ])

In [ ]:
### test corr and error of bay vs freq
sel_idx, metrics = evaluate_feature_selection_method_cls(
        baycofe_select_features_cls,  # now Bayesian version
        X_tr, y_tr, X_te, y_te,
        selector_params={'estimator': bayes_logit, 'evaluator': bayes_logit, 'slack': 0.0001})
print("BayCOFE Selected:", sel_idx)
print("Acc:", metrics["accuracy"], "LogLoss:", metrics["log_loss"], "entropy:", metrics['mean_entropy'])


BayCOFE Selected: [ 1  5  8  9 10 11 12]
Acc: 0.9707602339181286 LogLoss: 0.1183181859231244 entropy: 0.03227859305894667


In [ ]:
# BayCOFE (logistic) code to evaluate accuracy and log loss
"""
sel_baycofe, metrics_baycofe = evaluate_feature_selection_method_cls(
    baycofe_select_features_cls,
    X_tr, y_tr, X_te, y_te,
    selector_params={'estimator': bayes_logit, 'slack': slack}
)
print("BayCOFE Selected:", sel_baycofe)
print("Acc:", metrics_baycofe["accuracy"], "LogLoss:", metrics_baycofe["log_loss"])
"""
# RFE (wrap estimator so it has coef_)
sel_rfe, metrics_rfe = evaluate_feature_selection_method_cls(
    rfe_select_features,
    X_tr, y_tr, X_te, y_te,
    selector_params={'estimator': bayes_logit}  # wrapper happens inside rfe_select_features
)
print("RFE Selected:", sel_rfe)
print("Acc:", metrics_rfe["accuracy"], "LogLoss:", metrics_rfe["log_loss"])

# SFS (forward)
sel_sfs, metrics_sfs = evaluate_feature_selection_method_cls(
    forward_select_features,
    X_tr, y_tr, X_te, y_te,
    selector_params={'estimator': bayes_logit}
)
print("SFS Selected:", sel_sfs)
print("Acc:", metrics_sfs["accuracy"], "LogLoss:", metrics_sfs["log_loss"])

# SBS (backward)
sel_sbs, metrics_sbs = evaluate_feature_selection_method_cls(
    sbs_select_features,
    X_tr, y_tr, X_te, y_te,
    selector_params={'estimator': bayes_logit}
)
print("SBS Selected:", sel_sbs)
print("Acc:", metrics_sbs["accuracy"], "LogLoss:", metrics_sbs["log_loss"])

# Bidirectional (SFFS)
sel_bidir, metrics_bidir = evaluate_feature_selection_method_cls(
    bidirectional_select_features,
    X_tr, y_tr, X_te, y_te,
    selector_params={'estimator': bayes_logit}
)
print("Bidirectional Selected:", sel_bidir)
print("Acc:", metrics_bidir["accuracy"], "LogLoss:", metrics_bidir["log_loss"])

RFE Selected: [0 1 2 3 4 5 6 7 8 9]
Acc: 0.6533333333333333 LogLoss: 0.6277289380382874
SFS Selected: [0 1 5 6 9]
Acc: 0.69 LogLoss: 0.6788275301195336
SBS Selected: [0 1 5 6]
Acc: 0.69 LogLoss: 0.6779116168605839
Bidirectional Selected: [0 1 5 6]
Acc: 0.69 LogLoss: 0.6779116168605839
